--- Day 7: No Space Left On Device ---

You can hear birds chirping and raindrops hitting leaves as the expedition proceeds. Occasionally, you can even hear much louder sounds in the distance; how big do the animals get out here, anyway?

The device the Elves gave you has problems with more than just its communication system. You try to run a system update:

$ system-update --please --pretty-please-with-sugar-on-top
Error: No space left on device

Perhaps you can delete some files to make space for the update?

You browse around the filesystem to assess the situation and save the resulting terminal output (your puzzle input). For example:

$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k

The filesystem consists of a tree of files (plain data) and directories (which can contain other directories or files). The outermost directory is called /. You can navigate around the filesystem, moving into or out of directories and listing the contents of the directory you're currently in.

Within the terminal output, lines that begin with $ are commands you executed, very much like some modern computers:

    cd means change directory. This changes which directory is the current directory, but the specific result depends on the argument:
        cd x moves in one level: it looks in the current directory for the directory named x and makes it the current directory.
        cd .. moves out one level: it finds the directory that contains the current directory, then makes that directory the current directory.
        cd / switches the current directory to the outermost directory, /.
    ls means list. It prints out all of the files and directories immediately contained by the current directory:
        123 abc means that the current directory contains a file named abc with size 123.
        dir xyz means that the current directory contains a directory named xyz.

Given the commands and output in the example above, you can determine that the filesystem looks visually like this:

- / (dir)
  - a (dir)
    - e (dir)
      - i (file, size=584)
    - f (file, size=29116)
    - g (file, size=2557)
    - h.lst (file, size=62596)
  - b.txt (file, size=14848514)
  - c.dat (file, size=8504156)
  - d (dir)
    - j (file, size=4060174)
    - d.log (file, size=8033020)
    - d.ext (file, size=5626152)
    - k (file, size=7214296)

Here, there are four directories: / (the outermost directory), a and d (which are in /), and e (which is in a). These directories also contain files of various sizes.

Since the disk is full, your first step should probably be to find directories that are good candidates for deletion. To do this, you need to determine the total size of each directory. The total size of a directory is the sum of the sizes of the files it contains, directly or indirectly. (Directories themselves do not count as having any intrinsic size.)

The total sizes of the directories above can be found as follows:

    The total size of directory e is 584 because it contains a single file i of size 584 and no other directories.
    The directory a has total size 94853 because it contains files f (size 29116), g (size 2557), and h.lst (size 62596), plus file i indirectly (a contains e which contains i).
    Directory d has total size 24933642.
    As the outermost directory, / contains every file. Its total size is 48381165, the sum of the size of every file.

To begin, find all of the directories with a total size of at most 100000, then calculate the sum of their total sizes. In the example above, these directories are a and e; the sum of their total sizes is 95437 (94853 + 584). (As in this example, this process can count files more than once!)

Find all of the directories with a total size of at most 100000. What is the sum of the total sizes of those directories?

In [1]:
import scala.io.Source

val filename = "day7input"

val rawData = Source.fromFile(filename).getLines.toList

import scala.io.Source


filename: String = "day7input"
rawData: List[String] = List(
  "$ cd /",
  "$ ls",
  "dir bsnqsfm",
  "dir dtqvbspj",
  "dir hhhtrws",
  "dir ldmsq",
  "307337 pnm.slh",
  "dir pqcndb",
  "dir pwtqzwv",
  "212421 zcrfndg.cms",
  "$ cd bsnqsfm",
  "$ ls",
  "179236 lccnhn",
  "$ cd ..",
  "$ cd dtqvbspj",
  "$ ls",
  "221336 gdjfp.mfp",
  "273114 jjgpvcqv.jlq",
  "$ cd ..",
  "$ cd hhhtrws",
  "$ ls",
  "dir gcbg",
  "dir jjgpvcqv",
  "dir mgvdbtl",
  "1606 mgvdbtl.ztt",
  "dir qhv",
  "27538 wprqtd.wph",
  "$ cd gcbg",
  "$ ls",
  "dir bgcwh",
  "dir bsnqsfm",
  "dir jjgpvcqv",
  "186683 lccnhn",
  "dir rqbnd",
  "32944 zjfs.mdf",
  "$ cd bgcwh",
  "$ ls",
  "211273 nvns",
...

In [2]:
abstract class Command

case class Cd(dir: String) extends Command

case class CdUp() extends Command

case class CdRoot() extends Command

case class Ls() extends Command

abstract class Store

case class Dir(name: String, var items: Map[String, Store], parent: Option[Dir]) extends Store

case class File(name: String, size: Int) extends Store

defined class Command
defined class Cd
defined class CdUp
defined class CdRoot
defined class Ls
defined class Store
defined class Dir
defined class File

In [3]:
val lsRegex = """^\$ ls$""".r
val cdUpRegex = """^\$ cd ..$""".r
val cdRootRegex = """^\$ cd \/$""".r
val cdRegex = """^\$ cd (.*)$""".r
val dirRegex = """^dir (.*)$""".r
val fileRegex = """^(\d*) (.*)$""".r

def mapString(curDir: Dir, value: String) = value match {
    case lsRegex() =>  Ls()
    case cdRootRegex() => CdRoot()
    case cdUpRegex() => CdUp()
    case cdRegex(dir) => Cd(dir)
    case dirRegex(dir) => Dir(dir, Map.empty[String, Store], Some(curDir))
    case fileRegex(size, name) => File(name, size.toInt)
}

lsRegex: scala.util.matching.Regex = ^\$ ls$
cdUpRegex: scala.util.matching.Regex = ^\$ cd ..$
cdRootRegex: scala.util.matching.Regex = ^\$ cd \/$
cdRegex: scala.util.matching.Regex = ^\$ cd (.*)$
dirRegex: scala.util.matching.Regex = ^dir (.*)$
fileRegex: scala.util.matching.Regex = ^(\d*) (.*)$
defined function mapString

In [4]:
val root = new Dir("/", Map.empty[String, Store], None)

var curDir: Dir = root

var size = 0

rawData.foreach(
    stepString => {
        val step = mapString(curDir, stepString)
        step match {
            case Cd(dir) => curDir = curDir.items(dir) match {
                case dir: Dir => dir
            }
            case _: CdUp => curDir = curDir.parent.get
            case _: CdRoot => curDir = root
            case _: Ls => ()
            case dir: Dir => curDir.items += (dir.name -> dir)
            case file: File => curDir.items += (file.name -> file)
        }
    }
)


root: Dir = Dir(
  name = "/",
  items = HashMap(
    "ldmsq" -> Dir(
      name = "ldmsq",
      items = HashMap(
        "psgrjgr" -> Dir(
          name = "psgrjgr",
          items = HashMap(
            "dtqvbspj.hjw" -> File(name = "dtqvbspj.hjw", size = 283186),
            "fcvqsp" -> Dir(
              name = "fcvqsp",
              items = Map(
                "fzqmqlvs.hsc" -> File(name = "fzqmqlvs.hsc", size = 209758)
              ),
              parent = Some(
                value = Dir(
                  name = "psgrjgr",
                  items = HashMap(
                    "dtqvbspj.hjw" -> File(name = "dtqvbspj.hjw", size = 283186),
                    "fcvqsp" -> Dir(
                      name = "fcvqsp",
                      items = Map(
                        "fzqmqlvs.hsc" -> File(name = "fzqmqlvs.hsc", size = 209758)
                      ),
                      parent = Some(
                        value = Dir(
                          name = "psgrjgr",
                          items = HashMap(
                            "dtqvbspj.hjw" -> File(name = "dtqvbspj.hjw", size = 283186),
                            "fcvqsp" -> Dir(
                              name = "fcvqsp",
                              items = Map(
                                "fzqmqlvs.hsc" -> File(name = "fzqmqlvs.hsc", size = 209758)
                              ),
...
curDir: Dir = Dir(
  name = "zzclqz",
  items = Map("vcq.flg" -> File(name = "vcq.flg", size = 217719)),
  parent = Some(
    value = Dir(
      name = "pwtqzwv",
      items = HashMap(
        "zzclqz" -> Dir(
          name = "zzclqz",
          items = Map("vcq.flg" -> File(name = "vcq.flg", size = 217719)),
          parent = Some(
            value = Dir(
              name = "pwtqzwv",
              items = HashMap(
                "zzclqz" -> Dir(
                  name = "zzclqz",
                  items = Map(
                    "vcq.flg" -> File(name = "vcq.flg", size = 217719)
                  ),
                  parent = Some(
                    value = Dir(
                      name = "pwtqzwv",
                      items = HashMap(
                        "zzclqz" -> Dir(
                          name = "zzclqz",
                          items = Map(
                            "vcq.flg" -> File(name = "vcq.flg", size = 217719)
                          ),
                          parent = Some(
                            value = Dir(
                              name = "pwtqzwv",
                              items = HashMap(
                                "zzclqz" -> Dir(
                                  name = "zzclqz",
                                  items = Map(
                                    "vcq.flg" -> File(name = "vcq.flg", size = 217719)
                                  ),
                                  parent = Some(
...
size: Int = 0

In [5]:

var dirsFiles: Map[String, List[File]] = Map()

def sumSizes(dir: Dir, path: String): List[File] = {
    val (files: List[File], dirs: List[Dir]) = dir.items.values.partition(_.isInstanceOf[File])
    if (dirs.isEmpty) {
        dirsFiles += (path -> files)
        files
    } else {
        val dirFiles = dirs.flatMap(
            dir => {
                sumSizes(dir, path + dir.name + "/")
            }
        ) ++ files
        dirsFiles += (path -> dirFiles)
        dirFiles
    }
}

val rootDir = sumSizes(root, "/")

dirsFiles: Map[String, List[File]] = HashMap(
  "/pwtqzwv/jjgpvcqv/" -> List(
    File(name = "bsnqsfm.zjh", size = 149865),
    File(name = "jjgpvcqv.lwq", size = 1714),
    File(name = "gpcpgfh.hdd", size = 275833),
    File(name = "dtqvbspj.svq", size = 285861),
    File(name = "bsnqsfm", size = 14359),
    File(name = "dpqt.lph", size = 77969),
    File(name = "mgvdbtl", size = 15497),
    File(name = "hhsfwctl.hng", size = 156597)
  ),
  "/pqcndb/gpcpgfh/rbh/cwgc/" -> List(File(name = "ghsbvcw.wgh", size = 254910)),
  "/ldmsq/rtsmnzm/sdwmbsz/jbnsvcv/qfmj/bfvwng/qzdpp/" -> List(
    File(name = "rchw.lgb", size = 207083)
  ),
  "/pqcndb/zwph/cmznd/zhhm/" -> List(File(name = "rjh.qjf", size = 204738)),
  "/pwtqzwv/hthq/dtqvbspj/lrvftwzb/" -> List(
    File(name = "rcsffv.gbt", size = 92705),
    File(name = "wvthcng.qlf", size = 33209)
  ),
  "/pwtqzwv/jcsmcf/" -> List(File(name = "jjgpvcqv.jvj", size = 146144)),
  "/pqcndb/gpcpgfh/rbh/hdmmgms/gpcpgfh/" -> List(
    File(name = "jrs.bdw", size = 123990),
    File(name = "mgvdbtl", size = 52381),
    File(name = "rrqp.pmz", size = 130292)
  ),
  "/pqcndb/jjgpvcqv/rtnnjct/dtqvbspj/mgvdbtl/" -> List(
    File(name = "mgvdbtl.zhd", size = 115199),
    File(name = "pbqqz.dvg", size = 307710),
    File(name = "cqfppfg.cwl", size = 99356),
    File(name = "lccnhn", size = 192704),
    File(name = "qsv.nzw", size = 49374)
  ),
  "/ldmsq/rtsmnzm/tjwht/mgvdbtl/" -> List(
    File(name = "jvs.gvw", size = 61244),
    File(name = "rcsffv.gbt", size = 269474)
  ),
  "/hhhtrws/jjgpvcqv/jjgpvcqv/ptbsgmlr/" -> List(
...
defined function sumSizes
rootDir: List[File] = List(
  File(name = "fzqmqlvs.hsc", size = 209758),
  File(name = "dtqvbspj.hjw", size = 283186),
  File(name = "rcsffv.gbt", size = 258194),
  File(name = "jrs.bdw", size = 104691),
  File(name = "lccnhn", size = 286657),
  File(name = "nngdwngf.jpm", size = 189234),
  File(name = "qbq", size = 102320),
  File(name = "dhbfr", size = 168631),
  File(name = "jrr.jnj", size = 74345),
  File(name = "rchw.lgb", size = 207083),
  File(name = "dqmb", size = 273755),
  File(name = "lsgnjrn", size = 20840),
  File(name = "lvtbqlh", size = 83537),
  File(name = "jrs.bdw", size = 19558),
  File(name = "lccnhn", size = 72791),
  File(name = "pnm.slh", size = 262803),
  File(name = "sbdz", size = 40575),
  File(name = "jrr.jnj", size = 174151),
  File(name = "dtqvbspj.rww", size = 190922),
  File(name = "mwvpgtzm", size = 293765),
  File(name = "lccnhn", size = 59784),
  File(name = "jjgpvcqv.cvz", size = 140827),
  File(name = "jvs.gvw", size = 61244),
  File(name = "rcsffv.gbt", size = 269474),
  File(name = "rcsffv.gbt", size = 136586),
  File(name = "rzfvcjvh", size = 242800),
  File(name = "lccnhn", size = 217225),
  File(name = "gpcpgfh.zht", size = 105894),
  File(name = "nccmrdjv.tml", size = 20657),
  File(name = "nrb.zrj", size = 241440),
  File(name = "ppdh", size = 293894),
  File(name = "rcsffv.gbt", size = 310316),
  File(name = "czhbvv.fvc", size = 82960),
  File(name = "dtqvbspj.mqp", size = 238189),
  File(name = "lccnhn", size = 232175),
  File(name = "bnbvjg.nfq", size = 98034),
  File(name = "jrs.bdw", size = 99045),
  File(name = "cfnptz", size = 64751),
...

In [6]:
val dirSizes = dirsFiles.map{case (key, files) => (key, files.map(_.size).sum)}

dirSizes: Map[String, Int] = HashMap(
  "/pwtqzwv/jjgpvcqv/" -> 977695,
  "/pqcndb/gpcpgfh/rbh/cwgc/" -> 254910,
  "/ldmsq/rtsmnzm/sdwmbsz/jbnsvcv/qfmj/bfvwng/qzdpp/" -> 207083,
  "/pqcndb/zwph/cmznd/zhhm/" -> 204738,
  "/pwtqzwv/hthq/dtqvbspj/lrvftwzb/" -> 125914,
  "/pwtqzwv/jcsmcf/" -> 146144,
  "/pqcndb/gpcpgfh/rbh/hdmmgms/gpcpgfh/" -> 306663,
  "/pqcndb/jjgpvcqv/rtnnjct/dtqvbspj/mgvdbtl/" -> 764343,
  "/ldmsq/rtsmnzm/tjwht/mgvdbtl/" -> 330718,
  "/hhhtrws/jjgpvcqv/jjgpvcqv/ptbsgmlr/" -> 239257,
  "/pqcndb/gpcpgfh/rbh/hdmmgms/tgqbtw/" -> 460625,
  "/ldmsq/wsd/gpcpgfh/" -> 460025,
  "/pqcndb/jjgpvcqv/drmmqldt/jjgpvcqv/lptnp/" -> 150154,
  "/pqcndb/jjgpvcqv/trgbfqb/" -> 219127,
  "/pqcndb/jjgpvcqv/trgbfqb/mhhbp/hvjtmr/" -> 78682,
  "/pqcndb/jjgpvcqv/rtnnjct/ftlr/fdm/" -> 439951,
  "/hhhtrws/jjgpvcqv/" -> 6259122,
  "/pqcndb/pftswtnl/swgdh/mgvdbtl/" -> 880115,
  "/pqcndb/gpcpgfh/cmg/vsjs/ghw/" -> 394357,
  "/ldmsq/rtsmnzm/sdwmbsz/jjgpvcqv/vtbln/" -> 395727,
  "/pqcndb/pftswtnl/mgvdbtl

In [7]:
val smallDirs = dirSizes.values.filter(_ <= 100000)

val sumSmallDirs = smallDirs.sum

smallDirs: Iterable[Int] = List(
  78682,
  64193,
  69589,
  60362,
  99356,
  12412,
  61301,
  99356,
  61268,
  70984,
  32987,
  74345,
  37713,
  98034,
  60362,
  24916,
  11060,
  25816,
  12412,
  61301,
  62647,
  43065,
  80555,
  62647,
  91215,
  9385,
  51636
)
sumSmallDirs: Int = 1517599

--- Part Two ---

Now, you're ready to choose a directory to delete.

The total disk space available to the filesystem is 70000000. To run the update, you need unused space of at least 30000000. You need to find a directory you can delete that will free up enough space to run the update.

In the example above, the total size of the outermost directory (and thus the total amount of used space) is 48381165; this means that the size of the unused space must currently be 21618835, which isn't quite the 30000000 required by the update. Therefore, the update still requires a directory with total size of at least 8381165 to be deleted before it can run.

To achieve this, you have the following options:

    Delete directory e, which would increase unused space by 584.
    Delete directory a, which would increase unused space by 94853.
    Delete directory d, which would increase unused space by 24933642.
    Delete directory /, which would increase unused space by 48381165.

Directories e and a are both too small; deleting them would not free up enough space. However, directories d and / are both big enough! Between these, choose the smallest: d, increasing unused space by 24933642.

Find the smallest directory that, if deleted, would free up enough space on the filesystem to run the update. What is the total size of that directory?


In [8]:
val freeSpace = 70000000 - dirSizes("/")
val requiredToDelete = 30000000 - freeSpace

freeSpace: Int = 27523141
requiredToDelete: Int = 2476859

In [9]:
val dirSizesSorted = dirSizes.values.toList.sorted

dirSizesSorted: List[Int] = List(
  9385,
  11060,
  12412,
  12412,
  24916,
  25816,
  32987,
  37713,
  43065,
  51636,
  60362,
  60362,
  61268,
  61301,
  61301,
  62647,
  62647,
  64193,
  69589,
  70984,
  74345,
  78682,
  80555,
  91215,
  98034,
  99356,
  99356,
  108607,
  115949,
  115949,
  116833,
  118001,
  119862,
  119862,
  121073,
  125914,
  134541,
  142184,
...

In [10]:
val smallestDeletable = dirSizesSorted.find(_ >= requiredToDelete)

smallestDeletable: Option[Int] = Some(value = 2481982)